# Query-Passage差分を考慮した超平面実験

## 背景

e5-largeモデルでは、検索時に以下のプレフィックスを使い分ける：
- ドキュメント: `passage: {text}`
- クエリ: `query: {text}`

これにより検索精度が向上するが、LSHの超平面は `passage:` 同士の差分から生成されているため、
`query:` 埋め込みとの相関が弱くなっている可能性がある。

## 実験内容

### アプローチ1: Query-Passage差分超平面
同じテキストの `query:` と `passage:` 埋め込みの差分ベクトルを超平面に混ぜる。
これにより query→passage 方向の変換を直接捉える。

### アプローチ2: クエリもpassage:で埋め込み
検索時もpassage:プレフィックスを使う。e5の推奨とは異なるが、
同じ空間にマップされるためLSHの相関が保たれる可能性がある。

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
from numpy.linalg import norm
import pandas as pd
import duckdb
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

from src.lsh import SimHashGenerator, hamming_distance

## 1. データとモデルの準備

In [ ]:
# DuckDBに接続
con = duckdb.connect('../data/experiment_400k.duckdb', read_only=True)

# データ件数を確認
result = con.execute("SELECT dataset, COUNT(*) as cnt FROM documents GROUP BY dataset ORDER BY dataset").fetchall()
print('データセット別件数:')
for dataset, cnt in result:
    print(f'  {dataset}: {cnt:,}件')

In [ ]:
# E5モデルを読み込み
print('E5モデルを読み込み中...')
model = SentenceTransformer('intfloat/multilingual-e5-large')
print('読み込み完了')

In [ ]:
# 全データを読み込み（評価用）
print('データ読み込み中...')

datasets = ['body_en', 'body_ja', 'titles_en', 'titles_ja']
all_embeddings = {}
all_texts = {}
all_ids = {}

for dataset in tqdm(datasets, desc='データセット'):
    df = con.execute(f"""
        SELECT id, text, embedding 
        FROM documents 
        WHERE dataset = '{dataset}'
        ORDER BY id
    """).fetchdf()
    
    embeddings = np.array(df['embedding'].tolist(), dtype=np.float32)
    
    all_embeddings[dataset] = embeddings
    all_texts[dataset] = df['text'].values
    all_ids[dataset] = df['id'].values
    
    print(f'  {dataset}: {len(embeddings):,}件')

print('\n読み込み完了')

## 2. Query-Passage差分の分析

まず、同じテキストに対する `query:` と `passage:` 埋め込みの差を確認する。

In [ ]:
# サンプルテキストで query と passage の埋め込み差を確認
sample_texts = [
    '東京',
    '人工知能',
    'Tokyo is the capital of Japan',
    '日本の首都である東京は、世界最大の都市圏を持つ大都市です。',
    'Artificial intelligence is transforming industries worldwide.',
]

print('Query vs Passage 埋め込みの比較')
print('=' * 80)

for text in sample_texts:
    query_emb = model.encode(f'query: {text}', normalize_embeddings=False)
    passage_emb = model.encode(f'passage: {text}', normalize_embeddings=False)
    
    # コサイン類似度
    cos_sim = np.dot(query_emb, passage_emb) / (norm(query_emb) * norm(passage_emb))
    
    # L2距離
    l2_dist = norm(query_emb - passage_emb)
    
    # 差分ベクトルのノルム
    diff_norm = norm(query_emb - passage_emb)
    
    print(f'\nテキスト: {text[:50]}...' if len(text) > 50 else f'\nテキスト: {text}')
    print(f'  cos_sim(query, passage): {cos_sim:.4f}')
    print(f'  L2距離: {l2_dist:.4f}')

## 3. アプローチ1: Query-Passage差分超平面

超平面に query→passage 方向の差分ベクトルを混ぜる。

In [ ]:
def generate_query_passage_diff_hyperplanes(
    texts: list,
    model: SentenceTransformer,
    num_hyperplanes: int,
    seed: int = 42
) -> np.ndarray:
    """
    同じテキストの query: と passage: 埋め込みの差分を超平面として生成
    
    Args:
        texts: テキストリスト
        model: SentenceTransformerモデル
        num_hyperplanes: 生成する超平面数
        seed: 乱数シード
    
    Returns:
        超平面 (num_hyperplanes, dim)
    """
    rng = np.random.default_rng(seed)
    
    # テキストをサンプリング
    selected_indices = rng.choice(len(texts), min(num_hyperplanes, len(texts)), replace=False)
    selected_texts = [texts[i] for i in selected_indices]
    
    # query と passage の埋め込みを生成
    query_texts = [f'query: {t}' for t in selected_texts]
    passage_texts = [f'passage: {t}' for t in selected_texts]
    
    query_embs = model.encode(query_texts, normalize_embeddings=False, show_progress_bar=True)
    passage_embs = model.encode(passage_texts, normalize_embeddings=False, show_progress_bar=True)
    
    # 差分ベクトルを計算（query - passage）
    diff_vectors = query_embs - passage_embs
    
    # 正規化
    norms = np.linalg.norm(diff_vectors, axis=1, keepdims=True)
    hyperplanes = diff_vectors / norms
    
    return hyperplanes.astype(np.float32)

In [ ]:
def generate_data_sampled_hyperplanes(
    embeddings: np.ndarray,
    num_hyperplanes: int,
    seed: int = 42
) -> np.ndarray:
    """
    データの差分ベクトルから超平面を生成（従来手法）
    """
    rng = np.random.default_rng(seed)
    hyperplanes = []
    
    for _ in range(num_hyperplanes):
        i, j = rng.choice(len(embeddings), 2, replace=False)
        diff = embeddings[i] - embeddings[j]
        diff = diff / np.linalg.norm(diff)
        hyperplanes.append(diff)
    
    return np.array(hyperplanes, dtype=np.float32)

In [ ]:
# サンプルテキストを準備（body_jaから300件）
rng = np.random.default_rng(42)
sample_indices = rng.choice(len(all_texts['body_ja']), 300, replace=False)
sample_texts = [all_texts['body_ja'][i] for i in sample_indices]
sample_embeddings = all_embeddings['body_ja'][sample_indices]

print(f'サンプルテキスト数: {len(sample_texts)}')

In [ ]:
# 混合超平面パターンを定義
# 128ビット中の配分：DataSampled / QueryPassageDiff / Random

patterns = {
    'Baseline_DS': (128, 0, 0),      # 従来のDataSampledのみ
    'QP32': (96, 32, 0),              # DataSampled + QueryPassage差分32ビット
    'QP64': (64, 64, 0),              # DataSampled + QueryPassage差分64ビット
    'QP96': (32, 96, 0),              # DataSampled + QueryPassage差分96ビット
    'QP128': (0, 128, 0),             # QueryPassage差分のみ
    'QP64_R32': (32, 64, 32),         # 混合: DS32 + QP64 + Random32
}

print('実験パターン:')
for name, (ds, qp, r) in patterns.items():
    print(f'  {name}: DataSampled={ds}, QueryPassageDiff={qp}, Random={r}')

In [ ]:
# 各パターンの超平面を生成
print('超平面を生成中...')
print('=' * 80)

hyperplanes_patterns = {}

# まず各種の超平面を生成（最大数）
print('\n1. DataSampled超平面を生成...')
hp_datasampled = generate_data_sampled_hyperplanes(sample_embeddings, 128, seed=42)
print(f'  形状: {hp_datasampled.shape}')

print('\n2. QueryPassage差分超平面を生成...')
hp_qp_diff = generate_query_passage_diff_hyperplanes(sample_texts, model, 128, seed=42)
print(f'  形状: {hp_qp_diff.shape}')

print('\n3. ランダム超平面を生成...')
rng = np.random.default_rng(42)
hp_random = rng.standard_normal((128, 1024)).astype(np.float32)
hp_random = hp_random / np.linalg.norm(hp_random, axis=1, keepdims=True)
print(f'  形状: {hp_random.shape}')

# パターンごとに組み合わせ
print('\n4. 各パターンを組み合わせ...')
for name, (num_ds, num_qp, num_r) in patterns.items():
    parts = []
    if num_ds > 0:
        parts.append(hp_datasampled[:num_ds])
    if num_qp > 0:
        parts.append(hp_qp_diff[:num_qp])
    if num_r > 0:
        parts.append(hp_random[:num_r])
    
    hyperplanes_patterns[name] = np.vstack(parts)
    print(f'  {name}: {hyperplanes_patterns[name].shape}')

print('\n完了')

## 4. アプローチ2: クエリもpassage:で埋め込み

検索時にもpassage:プレフィックスを使用する。

In [ ]:
# 検索ワード（30件）
search_queries = [
    # 日本語 短文クエリ - 10件
    ('東京', 'ja', 'short'),
    ('人工知能', 'ja', 'short'),
    ('日本の歴史', 'ja', 'short'),
    ('プログラミング', 'ja', 'short'),
    ('音楽', 'ja', 'short'),
    ('環境問題', 'ja', 'short'),
    ('宇宙探査', 'ja', 'short'),
    ('経済学', 'ja', 'short'),
    ('医療技術', 'ja', 'short'),
    ('文学作品', 'ja', 'short'),
    
    # 日本語 曖昧文クエリ - 10件
    ('最近話題になっている技術革新について知りたいのですが、何かありますか', 'ja', 'ambiguous'),
    ('日本の伝統的な文化や芸術に関する情報を探しています', 'ja', 'ambiguous'),
    ('環境に優しい持続可能な社会を実現するための取り組みとは', 'ja', 'ambiguous'),
    ('健康的な生活を送るために必要なことは何でしょうか', 'ja', 'ambiguous'),
    ('世界の政治情勢や国際関係についての最新動向を教えて', 'ja', 'ambiguous'),
    ('子供の教育において大切にすべきポイントは何ですか', 'ja', 'ambiguous'),
    ('スポーツやフィットネスに関するトレンドを知りたい', 'ja', 'ambiguous'),
    ('美味しい料理のレシピや食文化についての情報', 'ja', 'ambiguous'),
    ('旅行や観光に関するおすすめの場所はありますか', 'ja', 'ambiguous'),
    ('ビジネスや起業に関する成功のヒントを教えてください', 'ja', 'ambiguous'),
    
    # 英語 短文クエリ - 5件
    ('Tokyo', 'en', 'short'),
    ('Artificial intelligence', 'en', 'short'),
    ('World history', 'en', 'short'),
    ('Programming', 'en', 'short'),
    ('Climate change', 'en', 'short'),
    
    # 英語 曖昧文クエリ - 5件
    ('I want to learn about recent technological innovations', 'en', 'ambiguous'),
    ('Looking for information about traditional culture and arts', 'en', 'ambiguous'),
    ('What are sustainable approaches to environmental protection', 'en', 'ambiguous'),
    ('Tell me about the latest developments in space exploration', 'en', 'ambiguous'),
    ('What are the key factors for business success and entrepreneurship', 'en', 'ambiguous'),
]

query_texts = [q[0] for q in search_queries]
print(f'検索ワード数: {len(query_texts)}件')

In [ ]:
# 両方のプレフィックスで埋め込みを生成
print('クエリ埋め込みを生成中...')

# アプローチ1用: query: プレフィックス
query_texts_with_query_prefix = [f'query: {text}' for text in query_texts]
query_embeddings_query = model.encode(query_texts_with_query_prefix, normalize_embeddings=False)
query_embeddings_query = query_embeddings_query.astype(np.float32)
print(f'  query:プレフィックス: {query_embeddings_query.shape}')

# アプローチ2用: passage: プレフィックス
query_texts_with_passage_prefix = [f'passage: {text}' for text in query_texts]
query_embeddings_passage = model.encode(query_texts_with_passage_prefix, normalize_embeddings=False)
query_embeddings_passage = query_embeddings_passage.astype(np.float32)
print(f'  passage:プレフィックス: {query_embeddings_passage.shape}')

print('完了')

## 5. 評価

In [ ]:
# 全データを統合
print('全データを統合中...')

all_embeddings_flat = np.vstack([all_embeddings[d] for d in datasets])
all_datasets_flat = []
all_ids_flat = []

for dataset in datasets:
    all_datasets_flat.extend([dataset] * len(all_embeddings[dataset]))
    all_ids_flat.extend(all_ids[dataset])

print(f'統合完了: {all_embeddings_flat.shape}')

In [ ]:
def evaluate_recall(
    query_embeddings: np.ndarray,
    all_embeddings: np.ndarray,
    hyperplanes: np.ndarray,
    candidate_limit: int = 2000,
    top_k: int = 10
) -> list:
    """
    クエリに対するRecall@kを評価
    
    Returns:
        各クエリのRecall値リスト
    """
    # SimHashGeneratorを作成して超平面を設定
    gen = SimHashGenerator(dim=1024, hash_bits=128, seed=0, strategy='random')
    gen.hyperplanes = hyperplanes
    
    # 全ドキュメントのハッシュを計算
    all_hashes = gen.hash_batch(all_embeddings)
    
    # クエリのハッシュを計算
    query_hashes = gen.hash_batch(query_embeddings)
    
    recalls = []
    
    for i in range(len(query_embeddings)):
        query_emb = query_embeddings[i]
        query_hash = query_hashes[i]
        
        # Ground Truth（コサイン類似度Top-k）
        cos_sims = (all_embeddings @ query_emb) / (norm(all_embeddings, axis=1) * norm(query_emb))
        gt_indices = set(np.argsort(cos_sims)[::-1][:top_k])
        
        # LSH候補（ハミング距離Top-candidate_limit）
        distances = [(j, hamming_distance(h, query_hash)) for j, h in enumerate(all_hashes)]
        distances.sort(key=lambda x: x[1])
        candidates = set(idx for idx, _ in distances[:candidate_limit])
        
        recall = len(gt_indices & candidates) / top_k
        recalls.append(recall)
    
    return recalls

In [ ]:
# 評価実行
print('評価を実行中...')
print('=' * 100)

candidate_limits = [1000, 2000, 5000]
results = []

for pattern_name, hyperplanes in tqdm(hyperplanes_patterns.items(), desc='パターン'):
    for limit in candidate_limits:
        # アプローチ1: query:プレフィックス + 各種超平面
        recalls_query = evaluate_recall(
            query_embeddings_query,
            all_embeddings_flat,
            hyperplanes,
            candidate_limit=limit
        )
        
        for i, (query_text, lang, query_type) in enumerate(search_queries):
            results.append({
                'pattern': pattern_name,
                'query_prefix': 'query:',
                'candidate_limit': limit,
                'query': query_text,
                'lang': lang,
                'query_type': query_type,
                'recall': recalls_query[i]
            })
        
        # アプローチ2: passage:プレフィックス + Baseline_DSのみ
        if pattern_name == 'Baseline_DS':
            recalls_passage = evaluate_recall(
                query_embeddings_passage,
                all_embeddings_flat,
                hyperplanes,
                candidate_limit=limit
            )
            
            for i, (query_text, lang, query_type) in enumerate(search_queries):
                results.append({
                    'pattern': 'Baseline_DS_PassageQuery',
                    'query_prefix': 'passage:',
                    'candidate_limit': limit,
                    'query': query_text,
                    'lang': lang,
                    'query_type': query_type,
                    'recall': recalls_passage[i]
                })

df_results = pd.DataFrame(results)
print('\n完了')

In [ ]:
# 結果サマリー
print('=' * 100)
print('外部クエリ検索 Recall@10（30クエリ平均）')
print('=' * 100)

# パターン × 候補数別の平均Recall
pivot = df_results.groupby(['pattern', 'candidate_limit'])['recall'].mean().unstack()

# パターン順にソート
pattern_order = ['Baseline_DS', 'Baseline_DS_PassageQuery', 'QP32', 'QP64', 'QP96', 'QP128', 'QP64_R32']
pivot = pivot.reindex([p for p in pattern_order if p in pivot.index])

print(f'\n{"パターン":>30} | {"クエリ接頭辞":>12} | {"1000件":>10} | {"2000件":>10} | {"5000件":>10}')
print('-' * 90)

for pattern in pivot.index:
    row = pivot.loc[pattern]
    prefix = df_results[df_results['pattern'] == pattern]['query_prefix'].iloc[0]
    print(f'{pattern:>30} | {prefix:>12} | {row[1000]:>10.1%} | {row[2000]:>10.1%} | {row[5000]:>10.1%}')

In [ ]:
# クエリタイプ別の結果（候補2000件）
print('\n' + '=' * 100)
print('クエリタイプ別 Recall@10（候補2000件）')
print('=' * 100)

subset = df_results[df_results['candidate_limit'] == 2000]

print(f'\n{"パターン":>30} | {"JA短文":>10} | {"JA曖昧":>10} | {"EN短文":>10} | {"EN曖昧":>10}')
print('-' * 85)

for pattern in [p for p in pattern_order if p in pivot.index]:
    pattern_subset = subset[subset['pattern'] == pattern]
    
    ja_short = pattern_subset[(pattern_subset['lang'] == 'ja') & (pattern_subset['query_type'] == 'short')]['recall'].mean()
    ja_amb = pattern_subset[(pattern_subset['lang'] == 'ja') & (pattern_subset['query_type'] == 'ambiguous')]['recall'].mean()
    en_short = pattern_subset[(pattern_subset['lang'] == 'en') & (pattern_subset['query_type'] == 'short')]['recall'].mean()
    en_amb = pattern_subset[(pattern_subset['lang'] == 'en') & (pattern_subset['query_type'] == 'ambiguous')]['recall'].mean()
    
    print(f'{pattern:>30} | {ja_short:>10.1%} | {ja_amb:>10.1%} | {en_short:>10.1%} | {en_amb:>10.1%}')

## 6. 詳細分析: クエリ「東京」でのハミング距離確認

In [ ]:
# 「東京」クエリの詳細分析
print('=' * 100)
print('クエリ「東京」のGround Truth分析')
print('=' * 100)

test_idx = 0  # 「東京」

# コサイン類似度を計算（query:プレフィックス使用）
query_emb_q = query_embeddings_query[test_idx]
cos_sims_q = (all_embeddings_flat @ query_emb_q) / (norm(all_embeddings_flat, axis=1) * norm(query_emb_q))
gt_indices_q = np.argsort(cos_sims_q)[::-1][:10]

# コサイン類似度を計算（passage:プレフィックス使用）
query_emb_p = query_embeddings_passage[test_idx]
cos_sims_p = (all_embeddings_flat @ query_emb_p) / (norm(all_embeddings_flat, axis=1) * norm(query_emb_p))
gt_indices_p = np.argsort(cos_sims_p)[::-1][:10]

print('\nGround Truth比較:')
print(f'  query:プレフィックス時のTop-10: {list(gt_indices_q)}')
print(f'  passage:プレフィックス時のTop-10: {list(gt_indices_p)}')
print(f'  共通件数: {len(set(gt_indices_q) & set(gt_indices_p))}/10')

In [ ]:
# 各パターンでのハミング距離
print('\n各パターンでのGT Top-10のハミング距離 (query:プレフィックス使用時):')
print(f'{"GT#":>5} | {"cos_sim":>8} | {"dataset":>12} | ', end='')
for pattern in ['Baseline_DS', 'QP64', 'QP128']:
    print(f'{pattern:>15} | ', end='')
print()
print('-' * 90)

for rank, idx in enumerate(gt_indices_q):
    print(f'{rank+1:>5} | {cos_sims_q[idx]:>8.4f} | {all_datasets_flat[idx]:>12} | ', end='')
    
    for pattern in ['Baseline_DS', 'QP64', 'QP128']:
        gen = SimHashGenerator(dim=1024, hash_bits=128, seed=0, strategy='random')
        gen.hyperplanes = hyperplanes_patterns[pattern]
        
        doc_hash = gen.hash_batch(all_embeddings_flat[idx:idx+1])[0]
        query_hash = gen.hash_batch(query_emb_q.reshape(1, -1))[0]
        ham_dist = hamming_distance(doc_hash, query_hash)
        print(f'{ham_dist:>15} | ', end='')
    print()

## 7. 可視化

In [ ]:
import matplotlib.pyplot as plt

# 結果を可視化
fig, ax = plt.subplots(figsize=(12, 6))

patterns_to_plot = [p for p in pattern_order if p in pivot.index]
x = np.arange(len(patterns_to_plot))
width = 0.25

for i, limit in enumerate(candidate_limits):
    values = [pivot.loc[p, limit] for p in patterns_to_plot]
    ax.bar(x + i*width, values, width, label=f'{limit:,}件')

ax.axhline(y=0.90, color='gray', linestyle=':', label='目標90%')
ax.set_ylabel('Recall@10')
ax.set_title('外部クエリ検索 Recall@10（30クエリ平均）')
ax.set_xticks(x + width)
ax.set_xticklabels(patterns_to_plot, rotation=45, ha='right')
ax.legend(loc='upper right')
ax.set_ylim(0, 1.0)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../data/18_query_passage_hyperplanes.png', dpi=150, bbox_inches='tight')
plt.show()

print('グラフを data/18_query_passage_hyperplanes.png に保存しました')

## 8. 結論

In [ ]:
# 接続を閉じる
con.close()
print('DuckDB接続を閉じました')

## 結果サマリー

### アプローチ1: Query-Passage差分超平面

- query: と passage: の埋め込み差分を超平面に混ぜることで、query→passage方向の変換を捉える
- 結果: [実験後に記入]

### アプローチ2: クエリもpassage:で埋め込み

- 検索時もpassage:プレフィックスを使用し、同じ空間にマップ
- 結果: [実験後に記入]

### 考察

[実験後に記入]